In [11]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201218'
endDate = '20201225'

readPath = r'L:\orderLog\data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
print(dataPathLs)

rawOrderLog = []
for thisPath in dataPathLs:
    data = pickle.load(open(thisPath, 'rb'))
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

# readPath = r'\\192.168.10.28\equityTradeLogs'
# dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
# dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
# dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

# rawOrderLog = []
# for thisPath in dataPathLs:
#     data = pd.read_csv(thisPath)
#     data = data.rename(columns={'mdClockAtArrival': 'caamd'})
#     rawOrderLog += [data]
# rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog['accCode'] // 100)
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog['sdd'] = orderLog.groupby('order')['sdd'].transform('first')
orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')
df = orderLog.copy()


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
taking = orderLog[orderLog['ars'].isin([1, 7])]
making = orderLog[orderLog['ars'].isin([2, 3])]
else_orders = orderLog[~(orderLog['ars'].isin([1, 2, 3, 7]))]
display('orders with abnormal ars values')
display(else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False))
display(else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False))
# else_orders[else_orders['updateType'] == 0].groupby('ars')['date'].size().sort_values(ascending=False).reset_index()['ars'].values
# kk = else_orders[else_orders['updateType'] == 0].groupby('accCode')['date'].size().sort_values(ascending=False)
# kk[kk > 20].index.values
taking['directNum'] = taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].transform('nunique')

if len(taking[taking['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(taking[(taking['directNum'] != 1) & (taking['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    taking = taking[taking['directNum'] == 1]

assert((taking.groupby(['date', 'secid', 'vai', 'sdd'])['orderDirection1'].nunique() == 1).all() == True)
orderLog = pd.concat([taking, making]).sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'sdd'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))
display(checkLog[checkLog['status'] == 3].groupby('updateType')['order'].size())
orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
try:
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
except:
    print(orderLog[orderLog['mrsb90'] == '-'])
    orderLog = orderLog[orderLog['mrsb90'] != '-']
    orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
try:
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
except:
    print(orderLog[orderLog['mrss90'] == '-'])
    orderLog = orderLog[orderLog['mrss90'] != '-']
    orderLog['mrss90'] = orderLog['mrss90'].astype(float)
try:
    orderLog['aaa'] = orderLog['aaa'].astype(float)
except:
    print(orderLog[orderLog['aaa'] == '-'])
    orderLog = orderLog[orderLog['aaa'] != '-']
    orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

['L:\\orderLog\\data\\20201218.pkl' 'L:\\orderLog\\data\\20201221.pkl'
 'L:\\orderLog\\data\\20201222.pkl' 'L:\\orderLog\\data\\20201223.pkl'
 'L:\\orderLog\\data\\20201224.pkl' 'L:\\orderLog\\data\\20201225.pkl']


'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
255035,20201218,zs_52_09,5291,2000060,-1,1,38646.0,0,0,-1.0,7.333550e+05
273604,20201218,zs_66_01,6634,2000609,-1,1,50000.0,0,0,-1.0,1.812810e+10
275958,20201218,zs_54_01,5474,2000628,-1,7,41131.0,0,0,-1.0,-1.000000e+00
303936,20201218,zs_54_01,5474,2000861,-1,1,35892.0,0,0,-1.0,8.689052e+08
342078,20201218,zs_66_01,6634,2002109,-1,1,40362.0,0,0,-1.0,1.812807e+10
351704,20201218,zs_66_01,6634,2002173,-1,7,37261.0,0,0,-1.0,-1.000000e+00
364212,20201218,zs_64_01,6480,2002248,-1,1,38258.0,0,0,-1.0,6.735200e+04
444072,20201218,zs_64_01,6480,2002799,-1,1,47480.0,0,0,-1.0,1.006880e+05
452971,20201218,zs_96_08,9685,2002833,-1,7,39881.0,0,0,-1.0,-1.000000e+00
583118,20201218,zs_96_08,9685,2300439,-1,7,52153.0,0,0,-1.0,-1.000000e+00


There are orders with same internalId but different orderId other than accCode 8856 case
date      colo      accCode  secid    orderDirection  absOrderSize  internalId
20201225  zs_52_09  5291     2002212  0               0             -1.0          2
Name: orderId, dtype: int64


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

10. no cancellation within 1 sec
any cancellation within 1 sec


,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,resa,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


,date,secid,colo,accCode,updateType,vai,absOrderSize,orderDirection,mrstaat,mrstauc,sdd,ApplSeqNum,sta
93888,20201218,1600761,zt_52_04,526901,0,4924159.0,3200,1,NaN,1000.0,145648000.0,NaN,else
273293,20201218,2000609,zs_52_09,5225,0,6486800.0,14500,-1,NaN,NaN,145646280.0,NaN,else
338942,20201218,2002078,zs_52_08,5276,0,18227449.0,1600,-1,NaN,NaN,145645980.0,NaN,else
541808,20201218,2300274,zs_94_03,9448,0,40428371.0,1200,1,NaN,NaN,145609440.0,NaN,else
718329,20201221,1600064,zt_96_09,966501,0,77292.0,200,1,NaN,NaN,145616000.0,NaN,else
821619,20201221,1600869,zs_52_06,5332,0,12439081.0,16500,-1,2000.0,1000.0,133407000.0,0.0,else
821620,20201221,1600869,zs_52_06,5332,2,12439081.0,16500,-1,2000.0,1000.0,133407000.0,0.0,else
821621,20201221,1600869,zs_52_06,5332,4,12439081.0,16500,-1,2000.0,1000.0,133407000.0,0.0,else
821622,20201221,1600869,zs_52_06,5332,4,12439081.0,16500,-1,2000.0,1000.0,133407000.0,0.0,else
821623,20201221,1600869,zs_52_06,5332,4,12439081.0,16500,-1,2000.0,1000.0,133407000.0,0.0,else


mrstaat  mrstauc
1000.0   0.0        158075
3000.0   1000.0     552193
         2000.0      38202
         3000.0      14735
11000.0  10000.0     13681
         20000.0      1168
         30000.0       795
13000.0  11000.0    340592
         12000.0     10752
         13000.0      2147
         21000.0      9202
         22000.0      9912
         23000.0      1870
         31000.0      4303
         32000.0      3511
         33000.0      4234
Name: date, dtype: int64

In [40]:
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc']

Series([], Name: mrstauc, dtype: float64)

In [103]:
test[test['order'] == 168275][['updateType', 'clockAtArrival', 'date', 'secid', 'vai', 'ars', 'orderDirection', 'tradePrice', 'absOrderSize',
                                       'absFilledThisUpdate', 'absOrderSizeCumFilled', 'accCode', 'clockAtArrival', 'ApplSeqNum', 'sdd', 
                                       'mra100', 'mrb100', 'orderPrice', 'mrm']]

,updateType,clockAtArrival,date,secid,vai,ars,orderDirection,tradePrice,absOrderSize,absFilledThisUpdate,absOrderSizeCumFilled,accCode,clockAtArrival,ApplSeqNum,sdd,mra100,mrb100,orderPrice,mrm
0,0,1608257791442794,20201218,1600053,733708.0,3.0,-1,-1.0,500,0,0,9441,1608257791442794,0.0,101631000.0,2029.0,2026.0,20.29,-0.000308
1,2,1608257791567958,20201218,1600053,733708.0,3.0,-1,-1.0,500,0,0,9441,1608257791567958,0.0,101631000.0,2029.0,2026.0,20.29,-0.000308
2,1,1608257827751589,20201218,1600053,733708.0,3.0,-1,-1.0,500,0,0,9441,1608257827751589,0.0,101631000.0,2029.0,2027.0,20.29,-0.000280
3,3,1608257827879363,20201218,1600053,733708.0,3.0,-1,-1.0,500,0,0,9441,1608257827879363,0.0,101631000.0,2029.0,2027.0,20.29,-0.000280


In [102]:
add[add['order'] == 168275]

,date,innerSeq,ID,accCode,updateType,order,time,mta_60s,mtmPrice_60s,mta_180s,mtmPrice_180s,mta_300s,mtmPrice_300s
0,20201218.0,0.0,1600053,9441.0,0.0,168275.0,2020-12-18 10:16:31.000120,-0.000162,20.280,7.571415e-07,20.275,-0.000212,20.270
1,20201218.0,1.0,1600053,9441.0,3.0,168275.0,2020-12-18 10:17:07.436689,-0.000162,20.295,-2.285401e-05,20.270,-0.000185,20.285


In [74]:
orderLog[orderLog['order'] == 206771][['updateType', 'clockAtArrival', 'date', 'secid', 'vai', 'ars', 'orderDirection', 'tradePrice', 'absOrderSize',
                                       'absFilledThisUpdate', 'absOrderSizeCumFilled', 'accCode', 'clockAtArrival', 'ApplSeqNum', 'sdd', 
                                       'mra100', 'mrb100', 'orderPrice']]

,updateType,clockAtArrival,date,secid,vai,ars,orderDirection,tradePrice,absOrderSize,absFilledThisUpdate,absOrderSizeCumFilled,accCode,clockAtArrival,ApplSeqNum,sdd,mra100,mrb100,orderPrice
831734,0,1608616757718723,20201222,1600864,135705289.0,1.0,1,-1.00,31200,0,0,9551,1608616757718723,0.0,135919000.0,745.0,744.0,7.45
831735,2,1608616757751244,20201222,1600864,135705289.0,1.0,1,-1.00,31200,0,0,9551,1608616757751244,0.0,135919000.0,745.0,744.0,7.45
831736,4,1608616758095334,20201222,1600864,135705289.0,1.0,1,7.45,31200,17300,17300,9551,1608616758095334,0.0,135919000.0,745.0,744.0,7.45
831737,4,1608616758132474,20201222,1600864,135705289.0,1.0,1,7.45,31200,2900,20200,9551,1608616758132474,0.0,135919000.0,745.0,744.0,7.45
831738,4,1608616758132545,20201222,1600864,135705289.0,1.0,1,7.45,31200,400,20600,9551,1608616758132545,0.0,135919000.0,745.0,744.0,7.45
831739,4,1608616758513467,20201222,1600864,135705289.0,1.0,1,7.45,31200,7000,27600,9551,1608616758513467,0.0,135919000.0,745.0,744.0,7.45
831740,1,1608616758738620,20201222,1600864,135705289.0,1.0,1,-1.00,31200,0,27600,9551,1608616758738620,0.0,135919000.0,747.0,746.0,7.45
831741,3,1608616758921511,20201222,1600864,135705289.0,1.0,1,-1.00,31200,0,29300,9551,1608616758921511,0.0,135919000.0,747.0,746.0,7.45
831742,4,1608616759113864,20201222,1600864,135705289.0,1.0,1,7.45,31200,1700,29300,9551,1608616759113864,0.0,135919000.0,747.0,746.0,7.45


In [12]:
IF_group = pd.read_table(r'E:\stockGroup_IF.txt', header=None)
IC_group = pd.read_table(r'E:\stockGroup_IC.txt', header=None)
CSI1000_group = pd.read_table(r'E:\stockGroup_CSI1000.txt', header=None)
IF_group1 = IF_group.iloc[0, :].values[0][1:-2].split(', ')
IF_group2 = IF_group.iloc[1, :].values[0][1:-2].split(', ')
IC_group1 = IC_group.iloc[0, :].values[0][1:-2].split(', ')
IC_group2 = IC_group.iloc[1, :].values[0][1:-2].split(', ')
CSI1000_group1 = CSI1000_group.iloc[0, :].values[0][1:-2].split(', ')
CSI1000_group2 = CSI1000_group.iloc[1, :].values[0][1:-2].split(', ')
assert(len(IF_group1) == len(IF_group2))
assert(len(IC_group1) == len(IC_group2))
assert(len(CSI1000_group1) == len(CSI1000_group2))

test1 = orderLog[orderLog['secid'].isin(IF_group1 + IF_group2)]
test1['index_name'] = 'IF'
test1['group_tag'] = np.where(test1['secid'].isin(IF_group1), 1, 2)
display(test1[(test1['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test1[(test1['updateType'] == 0) & (test1['accCode'] == 9461)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test2 = orderLog[orderLog['secid'].isin(IC_group1 + IC_group2)]
test2['index_name'] = 'IC'
test2['group_tag'] = np.where(test2['secid'].isin(IC_group1), 1, 2)
display(test2[(test2['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test2[(test2['updateType'] == 0) & (test2['accCode'] == 9441)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test3 = orderLog[orderLog['secid'].isin(CSI1000_group1 + CSI1000_group2)]
test3['index_name'] = 'CSI1000'
test3['group_tag'] = np.where(test3['secid'].isin(CSI1000_group1), 1, 2)
display(test3[(test3['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test3[(test3['updateType'] == 0) & (test3['accCode'].isin([5456, 9758]))].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())

test = pd.concat([test2[test2['accCode'] == 9441], test3[test3['accCode'] == 5456], 
                  test3[(test3['accCode'] == 9758) & (test3['date'] == 20201221)], test1[(test1['accCode'] == 9461) & (test1['date'] >= 20201221)]])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


colo      accCode  group_tag
zs_52_06  5384     1                 [1.0]
                   2                 [1.0]
zs_52_08  5281     1                 [1.0]
                   2                 [1.0]
          5377     2                 [1.0]
          5386     2                 [1.0]
zs_52_09  5264     1                 [1.0]
                   2                 [1.0]
zs_88_04  8967     1                 [1.0]
                   2                 [1.0]
          892402   1                 [1.0]
                   2                 [1.0]
zs_94_03  9461     1            [2.0, 3.0]
                   2            [1.0, 7.0]
zs_94_05  9435     1                 [1.0]
                   2                 [1.0]
zs_96_02  9655     1                 [1.0]
                   2                 [1.0]
zs_96_06  9765     1                 [1.0]
                   2                 [1.0]
zs_96_08  9754     2                 [1.0]
          974102   2                 [1.0]
zt_88_06  8970     1     

colo      accCode  group_tag
zs_94_03  9461     1            38
                   2            31
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5332     1                 [1.0]
                   2                 [1.0]
          5384     1                 [1.0]
                   2                 [1.0]
zs_52_08  5229     1                 [1.0]
                   2                 [1.0]
          5276     1                 [1.0]
                   2                 [1.0]
          5292     1                 [1.0]
                   2                 [1.0]
          5329     1                 [1.0]
                   2                 [1.0]
          5377     1                 [1.0]
                   2                 [1.0]
          5386     1                 [1.0]
                   2                 [1.0]
zs_52_09  5289     1                 [1.0]
                   2                 [1.0]
          5291     1                 [1.0]
                   2                 [1.0]
zs_68_01  6878     1                 [1.0]
                   2                 [1.0]
zs_72_01  7293     1     

colo      accCode  group_tag
zt_94_06  9441     1            38
                   2            30
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5232     1                      [1.0]
                   2                      [1.0]
          5275     1                      [1.0]
                   2                      [1.0]
          5287     1                      [1.0]
                   2                      [1.0]
          5384     1                      [1.0]
                   2                      [1.0]
zs_52_08  5377     1                      [1.0]
                   2                      [1.0]
          5386     1                      [1.0]
                   2                      [1.0]
zs_54_01  5456     1                 [3.0, 2.0]
                   2                 [1.0, 7.0]
zs_66_01  6623     1                      [1.0]
                   2                      [1.0]
          6627     1                      [1.0]
                   2                      [1.0]
          6683     1                      [1.0]
                   2                      [1.0]
zs_88_04  8

colo      accCode  group_tag
zs_54_01  5456     1             98
                   2             96
zs_96_06  9758     1            111
                   2            105
Name: secid, dtype: int64

In [28]:
test.groupby(['date', 'accCode'])['ars'].unique()

date      accCode
20201218  5456            [1.0, 3.0, 2.0]
          9441       [3.0, 2.0, 1.0, 7.0]
20201221  5456       [1.0, 2.0, 3.0, 7.0]
          9441       [3.0, 2.0, 1.0, 7.0]
          9461            [3.0, 2.0, 1.0]
          9758            [1.0, 3.0, 2.0]
20201222  5456            [1.0, 3.0, 2.0]
          9441       [3.0, 2.0, 1.0, 7.0]
          9461            [3.0, 2.0, 1.0]
20201223  5456            [1.0, 3.0, 2.0]
          9441       [3.0, 2.0, 1.0, 7.0]
          9461            [3.0, 2.0, 1.0]
20201224  5456            [3.0, 2.0, 1.0]
          9441       [2.0, 3.0, 1.0, 7.0]
          9461       [3.0, 2.0, 1.0, 7.0]
20201225  5456            [3.0, 1.0, 2.0]
          9441       [3.0, 2.0, 1.0, 7.0]
          9461            [3.0, 1.0, 2.0]
Name: ars, dtype: object

In [17]:
test1 = test[test['date'] == 20201225].sort_values(by = ['date', 'colo', 'accCode', 'secid', 'vai', 'order', 'clockAtArrival'])
test1['sdd'] = test1.groupby(['order'])['sdd'].transform('first')
test1['datetime'] = (test1['date'] * 1000000000 + test1['sdd'].astype('int64')).astype('int64').astype(str)
test1['datetime'] = pd.to_datetime(test1['datetime'], format='%Y%m%d%H%M%S%f')
test1["start_time"] = test1['datetime'].apply(lambda x: np.int64((x + datetime.timedelta(hours=-8)).timestamp() * 1e6))
test1['caamd'] = test1.groupby('order')['caamd'].transform('first')
test1['mdClockAtArrival'] = test1['clockAtArrival'] - test1['caamd'] + test1['start_time']
test1['time'] = test1['mdClockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
test1 = test1[test1['updateType'].isin([0, 4, 3])]
test1 = test1.sort_values(by=['date', 'secid', 'vai', 'colo', 'accCode', 'order', 'clockAtArrival']).reset_index(drop=True)
test1['innerSeq'] = test1.index
data = test1[['date', 'innerSeq', 'secid', 'accCode', 'updateType', 'order', 'time']].reset_index(drop=True)
data

,date,innerSeq,secid,accCode,updateType,order,time
0,20201225,0,1600053,9441,0,1157007,2020-12-25 09:43:49.000064
1,20201225,1,1600053,9441,3,1157007,2020-12-25 09:44:03.866265
2,20201225,2,1600053,9441,0,1157008,2020-12-25 09:50:28.000074
3,20201225,3,1600053,9441,3,1157008,2020-12-25 09:51:40.140600
4,20201225,4,1600053,9441,0,1157009,2020-12-25 10:22:27.000060
...,...,...,...,...,...,...,...
6836,20201225,6836,2300815,5456,0,1028884,2020-12-25 09:38:02.530027
6837,20201225,6837,2300815,5456,3,1028884,2020-12-25 09:38:05.509338
6838,20201225,6838,2300815,5456,0,1028885,2020-12-25 09:50:53.270025
6839,20201225,6839,2300815,5456,4,1028885,2020-12-25 09:50:55.507176


In [29]:
import pandas as pd
import numpy as np
import pickle

sumup = []
for d in [20201221, 20201222, 20201223, 20201224]:
    add = pickle.load(open('\\\\192.168.10.34\\trading\\Kevin_Zhenyu_Kirk\\processed\\makingOrder' + str(d) +'.pkl', 'rb'))
    test1 = test[test['date'] == d].sort_values(by = ['date', 'colo', 'accCode', 'secid', 'vai', 'order', 'clockAtArrival'])
    test1['sdd'] = test1.groupby(['order'])['sdd'].transform('first')
    test1['datetime'] = (test1['date'] * 1000000000 + test1['sdd'].astype('int64')).astype('int64').astype(str)
    test1['datetime'] = pd.to_datetime(test1['datetime'], format='%Y%m%d%H%M%S%f')
    test1["start_time"] = test1['datetime'].apply(lambda x: np.int64((x + datetime.timedelta(hours=-8)).timestamp() * 1e6))
    test1['caamd'] = test1.groupby('order')['caamd'].transform('first')
    test1['mdClockAtArrival'] = test1['clockAtArrival'] - test1['caamd'] + test1['start_time']
    test1['time'] = test1['mdClockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    test1 = test1[test1['updateType'].isin([0, 4, 3])]
    test1 = test1.sort_values(by=['date', 'secid', 'vai', 'colo', 'accCode', 'order', 'clockAtArrival']).reset_index(drop=True)
    test1['innerSeq'] = test1.index
    display(add.shape[0])
    display(test1.shape[0])
    assert(add.shape[0] == test1.shape[0])
    test1 = pd.merge(test1[['order', 'innerSeq', 'clockAtArrival']], add[['order', 'innerSeq', 'mta_60s', 'mtmPrice_60s', 'mta_180s', 'mtmPrice_180s', 'mta_300s',
           'mtmPrice_300s']], on=['order', 'innerSeq'], how='left', validate='one_to_one')
    sumup += [test1]
sumup = pd.concat(sumup)
test = pd.merge(test, sumup[['order', 'clockAtArrival', 'mta_60s', 'mtmPrice_60s', 'mta_180s', 'mtmPrice_180s', 'mta_300s',
       'mtmPrice_300s']], on=['order', 'clockAtArrival'], how='left', validate='one_to_one')

9733

9733

5964

5964

5939

5939

7217

7217

In [127]:
test[test['updateType'].isin([0, 4, 3])].shape[0]

20516

In [128]:
pd.read_pickle(r'\\192.168.10.34\trading\Kevin_Zhenyu_Kirk\raw\20201218.pkl').shape[0] + \
pd.read_pickle(r'\\192.168.10.34\trading\Kevin_Zhenyu_Kirk\raw\20201221.pkl').shape[0] + \
pd.read_pickle(r'\\192.168.10.34\trading\Kevin_Zhenyu_Kirk\raw\20201222.pkl').shape[0]

20516

In [126]:
data.to_pickle(r'\\192.168.10.34\trading\Kevin_Zhenyu_Kirk\raw\20201222.pkl')

In [3]:
test[test['updateType'] == 0].groupby(['date', 'accCode', 'ars'])['orderPrice'].size()

date      accCode  ars
20201218  5456     1.0    333
                   2.0    238
                   3.0    435
          9441     1.0    291
                   2.0    326
                   3.0    483
                   7.0      3
20201221  5456     1.0    294
                   2.0    188
                   3.0    333
                   7.0     11
          9441     1.0    330
                   2.0    409
                   3.0    471
                   7.0      3
          9461     1.0    166
                   2.0    110
                   3.0    205
          9758     1.0    525
                   2.0    470
                   3.0    745
20201222  5456     1.0    359
                   2.0    220
                   3.0    452
          9441     1.0    315
                   2.0    255
                   3.0    481
                   7.0     12
          9461     1.0    230
                   2.0    100
                   3.0    159
Name: orderPrice, dtype: int64

In [60]:
rawOrderLog[rawOrderLog['accCode'].isin([9758, 9461]) & (rawOrderLog['updateType'] == 0)].groupby(['date', 'accCode', 'ars'])['orderPrice'].size()

date      accCode  ars
20201218  9461     0.0       2
                   1.0     192
                   2.0     163
          9758     1.0    1139
                   2.0     814
20201221  9461     0.0       3
                   1.0     204
                   2.0     110
                   3.0     205
          9758     1.0    1033
                   2.0     470
                   3.0     745
                   7.0      13
Name: orderPrice, dtype: int64

In [94]:
test1 = df[df['secid'].isin(IF_group1 + IF_group2)]
test2 = df[df['secid'].isin(IC_group1 + IC_group2)]
test3 = df[df['secid'].isin(CSI1000_group1 + CSI1000_group2)]
testt = pd.concat([test2[(test2['accCode'] == 9441)], test3[test3['accCode'].isin([5456, 9758])], test1[test1['accCode'] == 9461]])

test1 = testt[testt['ars'].isin([2, 3])].sort_values(by = ['date', 'colo', 'accCode', 'secid', 'vai', 'order', 'clockAtArrival'])
test1['sdd'] = test1.groupby(['order'])['sdd'].transform('first')
test1['datetime'] = (test1['date'] * 1000000000 + test1['sdd'].astype('int64')).astype('int64').astype(str)
test1['datetime'] = pd.to_datetime(test1['datetime'], format='%Y%m%d%H%M%S%f')
test1["start_time"] = test1['datetime'].apply(lambda x: np.int64((x + datetime.timedelta(hours=-8)).timestamp() * 1e6))
test1['caamd'] = test1.groupby('order')['caamd'].transform('first')
test1['mdClockAtArrival'] = test1['clockAtArrival'] - test1['caamd'] + test1['start_time']
test1['time'] = test1['mdClockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
test1 = test1[test1['updateType'].isin([0, 4, 3])]
test1 = test1.sort_values(by=['date', 'secid', 'vai', 'colo', 'accCode', 'order', 'clockAtArrival']).reset_index(drop=True)
test1['innerSeq'] = test1.index
data = test1[['date', 'innerSeq', 'secid', 'accCode', 'updateType', 'order', 'time']].reset_index(drop=True)
data

,date,innerSeq,secid,accCode,updateType,order,time
0,20201218,0,1600053,9441,0,168275,2020-12-18 10:16:31.000120
1,20201218,1,1600053,9441,3,168275,2020-12-18 10:17:07.436689
2,20201218,2,1600053,9441,0,168276,2020-12-18 10:18:17.000087
3,20201218,3,1600053,9441,3,168276,2020-12-18 10:19:11.197035
4,20201218,4,1600053,9441,0,168277,2020-12-18 10:26:26.000051
...,...,...,...,...,...,...,...
12630,20201221,12630,2300815,9758,3,266537,2020-12-21 13:16:52.291927
12631,20201221,12631,2300815,9758,0,266538,2020-12-21 14:00:15.510057
12632,20201221,12632,2300815,9758,3,266538,2020-12-21 14:00:51.377694
12633,20201221,12633,2300815,9758,0,266539,2020-12-21 14:53:13.570061


In [63]:
test[['updateType', 'clockAtArrival', 'date', 'secid', 'vai', 'ars', 'orderDirection', 'tradePrice', 'absOrderSize',
                                       'absFilledThisUpdate', 'absOrderSizeCumFilled', 'accCode', 'clockAtArrival', 'ApplSeqNum', 'sdd', 
                                       'mra100', 'mrb100', 'orderPrice', 'mta_60s', 'mtmPrice_60s', 'mta_180s', 'mtmPrice_180s', 'mta_300s',
       'mtmPrice_300s']]

,updateType,clockAtArrival,date,secid,vai,ars,orderDirection,tradePrice,absOrderSize,absFilledThisUpdate,absOrderSizeCumFilled,accCode,clockAtArrival,ApplSeqNum,sdd,mra100,mrb100,orderPrice,mta_60s,mtmPrice_60s,mta_180s,mtmPrice_180s,mta_300s,mtmPrice_300s
0,0,1608257791442794,20201218,1600053,733708.0,3.0,-1,-1.00,500,0,0,9441,1608257791442794,0.0,101631000.0,2029.0,2026.0,20.29,-0.000162,20.280,7.571415e-07,20.275,-0.000212,20.270
1,2,1608257791567958,20201218,1600053,733708.0,3.0,-1,-1.00,500,0,0,9441,1608257791567958,0.0,101631000.0,2029.0,2026.0,20.29,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1608257827751589,20201218,1600053,733708.0,3.0,-1,-1.00,500,0,0,9441,1608257827751589,0.0,101631000.0,2029.0,2027.0,20.29,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1608257827879363,20201218,1600053,733708.0,3.0,-1,-1.00,500,0,0,9441,1608257827879363,0.0,101631000.0,2029.0,2027.0,20.29,-0.000162,20.295,-2.285401e-05,20.270,-0.000185,20.285
4,0,1608257896910879,20201218,1600053,739708.0,3.0,-1,-1.00,500,0,0,9441,1608257896910879,0.0,101817000.0,2032.0,2030.0,20.32,0.000030,20.280,-3.754619e-04,20.270,0.000089,20.280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24430,2,1608514488974271,20201221,2300628,142050.0,1.0,-1,-1.00,200,0,0,9461,1608514488974271,0.0,93451080.0,6790.0,6789.0,67.89,NaN,NaN,NaN,NaN,NaN,NaN
24431,4,1608514488974335,20201221,2300628,142050.0,1.0,-1,67.89,200,200,200,9461,1608514488974335,0.0,93451080.0,6790.0,6789.0,67.89,NaN,NaN,NaN,NaN,NaN,NaN
24432,0,1608514704016499,20201221,2300628,220050.0,1.0,-1,-1.00,1800,0,0,9461,1608514704016499,2510600.0,93826120.0,6906.0,6900.0,69.00,NaN,NaN,NaN,NaN,NaN,NaN
24433,2,1608514704019808,20201221,2300628,220050.0,1.0,-1,-1.00,1800,0,0,9461,1608514704019808,0.0,93826120.0,6906.0,6900.0,69.00,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df[df['order'].isin([169501.0, 169752.0])][['updateType', 'clockAtArrival', 'date', 'secid', 'vai', 'ars', 'orderDirection', 'tradePrice', 'absOrderSize',
                                       'absFilledThisUpdate', 'absOrderSizeCumFilled', 'accCode', 'clockAtArrival', 'ApplSeqNum', 'sdd', 
                                       'mra100', 'mrb100', 'orderPrice']]

,updateType,clockAtArrival,date,secid,vai,ars,orderDirection,tradePrice,absOrderSize,absFilledThisUpdate,absOrderSizeCumFilled,accCode,clockAtArrival,ApplSeqNum,sdd,mra100,mrb100,orderPrice
683497,0,1608274586420568,20201218,1603355,892047.0,2.0,1,-1.00,400,0,0,9441,1608274586420568,0.0,145627000.0,3185.0,3175.0,31.76
683498,2,1608274586638302,20201218,1603355,892047.0,2.0,1,-1.00,400,0,0,9441,1608274586638302,0.0,145627000.0,3185.0,3175.0,31.76
683499,4,1608274590055851,20201218,1603355,892047.0,2.0,1,31.76,400,82,82,9441,1608274590055851,0.0,145627000.0,3184.0,3176.0,31.76
684521,0,1608274550647996,20201218,1603816,2056283.0,2.0,1,-1.00,600,0,0,9441,1608274550647996,0.0,145551000.0,7667.0,7645.0,76.46
684522,2,1608274550813303,20201218,1603816,2056283.0,2.0,1,-1.00,600,0,0,9441,1608274550813303,0.0,145551000.0,7667.0,7645.0,76.46
684523,4,1608274556909399,20201218,1603816,2056283.0,2.0,1,76.46,600,100,100,9441,1608274556909399,0.0,145551000.0,7666.0,7646.0,76.46
684524,4,1608274559788660,20201218,1603816,2056283.0,2.0,1,76.46,600,300,400,9441,1608274559788660,0.0,145551000.0,7665.0,7646.0,76.46
684525,4,1608274583460672,20201218,1603816,2056283.0,2.0,1,76.46,600,100,500,9441,1608274583460672,0.0,145551000.0,7665.0,7646.0,76.46


In [8]:
test.groupby('date')['accCode'].unique()

date
20201218                [9441, 5456]
20201221    [9441, 5456, 9758, 9461]
20201222          [9441, 5456, 9461]
Name: accCode, dtype: object

In [39]:
test = test[(test['ars'].isin([1, 2, 3])) & (test['date'] >= 20201221) & (test['date'] <= 20201224)]
# test['order_type'] = np.where(test['ars'] == 1, 'take', np.where(
# test['ars'] == 2, 'improved-NBBO make', 'on-NBBO make'))
test['order_type'] = np.where(test['ars'] == 1, 'take', 'make')
test['Side'] = np.where(test['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
test['duration'] = np.where(test['accCode'] == 9758, '20201221-20201221', '20201221-20201224')
f1 = test[(test['updateType'] == 0)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['orderNtl'].sum().reset_index()
f2 = test[(test['updateType'] == 4)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].sum().reset_index()
f = pd.merge(f1, f2, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
f['fill_rate'] = f['tradeNtl'] / f['orderNtl']
f1 = test[(test['updateType'] == 0)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['order'].nunique().reset_index()
f2 = test[(test['updateType'] == 3)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['order'].nunique().reset_index()
ff = pd.merge(f1, f2, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff['cancel_rate'] = ff['order_y'] / ff['order_x']
f = pd.merge(f, ff[['duration', 'Side', 'order_type', 'cancel_rate', 'accCode', 'index_name']], on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
test['cancel_duration_med'] = test['clockAtArrival'] - test['startClock']
test['cancel_duration_90p'] = test['clockAtArrival'] - test['startClock']
ff = test[test['updateType'] == 3].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['cancel_duration_med'].median().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff = test[test['updateType'] == 3].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['cancel_duration_90p'].quantile(0.9).reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
test1 = test[test['updateType'] == 4]
test1['mid'] = (test1['mra100'] + test1['mrb100'])/200
test1['tp'] = (test1['mrm'] + 1) * test1['mid']
test1['mrm_return'] = np.where(test1['Side'] == 'Sell', test1['tradePrice']/test1['tp'] - 1, test1['tp']/test1['tradePrice'] - 1)
test1['sumNtl'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
test1['sum_return'] = test1['mrm_return'] * test1['tradeNtl']
test1['sum_return'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
test1['mrm_return'] = test1['sum_return'] / test1['sumNtl']

test1['tp'] = (test1['mta_60s'] + 1) * test1['mtmPrice_60s']
test1['mrm_return_60s'] = np.where(test1['Side'] == 'Sell', test1['tradePrice']/test1['tp'] - 1, test1['tp']/test1['tradePrice'] - 1)
test1['sumNtl'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
test1['sum_return'] = test1['mrm_return_60s'] * test1['tradeNtl']
test1['sum_return'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
test1['mrm_return_60s'] = test1['sum_return'] / test1['sumNtl']

test1['tp'] = (test1['mta_180s'] + 1) * test1['mtmPrice_180s']
test1['mrm_return_180s'] = np.where(test1['Side'] == 'Sell', test1['tradePrice']/test1['tp'] - 1, test1['tp']/test1['tradePrice'] - 1)
test1['sumNtl'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
test1['sum_return'] = test1['mrm_return_180s'] * test1['tradeNtl']
test1['sum_return'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
test1['mrm_return_180s'] = test1['sum_return'] / test1['sumNtl']

test1['tp'] = (test1['mta_300s'] + 1) * test1['mtmPrice_300s']
test1['mrm_return_300s'] = np.where(test1['Side'] == 'Sell', test1['tradePrice']/test1['tp'] - 1, test1['tp']/test1['tradePrice'] - 1)
test1['sumNtl'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
test1['sum_return'] = test1['mrm_return_300s'] * test1['tradeNtl']
test1['sum_return'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
test1['mrm_return_300s'] = test1['sum_return'] / test1['sumNtl']

ff = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['mrm_return', 'mrm_return_60s', 'mrm_return_180s', 'mrm_return_300s'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
from IPython.display import display, HTML
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'order_type']).first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [28]:
test[(test['updateType'] == 4)].groupby('order_type')['tradeNtl'].sum()

order_type
improved-NBBO make    32689576.67
on-NBBO make          41640081.02
take                  53384169.99
Name: tradeNtl, dtype: float64

In [6]:
test[~test['mta_60s'].isnull()]['updateType'].unique()

array([0, 3, 4], dtype=int64)

In [40]:
rdf = test[test['updateType'] == 4]
if rdf[rdf['absFilledThisUpdate'] == 0].shape[0] != 0:
    print('There are stocks with zero trade size')
    print(rdf[rdf['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
    rdf = rdf[rdf['absFilledThisUpdate'] != 0]
if rdf[rdf['beta_60'].isnull()].shape[0] != 0:
    print('There are stocks with null beta')
    print(rdf[rdf['beta_60'].isnull()])
    rdf = rdf[~rdf['beta_60'].isnull()]
rdf["buyRet"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["adjMid_F90s"] / rdf["tradePrice"] - 1, np.nan)
rdf["buyRet1"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["adjMid_F300s"] / rdf["tradePrice"] - 1, np.nan)
rdf["sellRet"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["adjMid_F90s"] - 1, np.nan)
rdf["sellRet1"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["adjMid_F300s"] - 1, np.nan)

rdf['F90s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet'], rdf['sellRet'])
rdf['F300s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet1'], rdf['sellRet1'])

rdf['sumNtl'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
rdf['sum_return'] = rdf['F90s_return'] * rdf['tradeNtl']
rdf['sum_return'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
rdf['F90s_return'] = rdf['sum_return'] / rdf['sumNtl']
rdf['sum_return1'] = rdf['F300s_return'] * rdf['tradeNtl']
rdf['sum_return1'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return1'].transform(sum)
rdf['F300s_return'] = rdf['sum_return1'] / rdf['sumNtl']
ff = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['F90s_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['F300s_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'order_type']).first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [12]:
rdf = test[test['updateType'] == 4]
if rdf[rdf['absFilledThisUpdate'] == 0].shape[0] != 0:
    print('There are stocks with zero trade size')
    print(rdf[rdf['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
    rdf = rdf[rdf['absFilledThisUpdate'] != 0]
if rdf[rdf['beta_60'].isnull()].shape[0] != 0:
    print('There are stocks with null beta')
    print(rdf[rdf['beta_60'].isnull()])
    rdf = rdf[~rdf['beta_60'].isnull()]
rdf["buyRet"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["adjMid_F90s"] / rdf["tradePrice"] - 1, np.nan)
rdf["buyRet1"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["adjMid_F300s"] / rdf["tradePrice"] - 1, np.nan)
rdf["sellRet"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["adjMid_F90s"] - 1, np.nan)
rdf["sellRet1"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["adjMid_F300s"] - 1, np.nan)

rdf["buyRet2"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["mtmPrice_60s"] / rdf["tradePrice"] - 1, np.nan)
rdf["buyRet3"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["mtmPrice_180s"] / rdf["tradePrice"] - 1, np.nan)
rdf["buyRet4"] = np.where(rdf["orderDirection"].isin([1, 2]), rdf["mtmPrice_300s"] / rdf["tradePrice"] - 1, np.nan)
rdf["sellRet2"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["mtmPrice_60s"] - 1, np.nan)
rdf["sellRet3"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["mtmPrice_180s"] - 1, np.nan)
rdf["sellRet4"] = np.where(rdf["orderDirection"].isin([-1, -2]), rdf["tradePrice"] / rdf["mtmPrice_300s"] - 1, np.nan)

rdf['F90s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet'], rdf['sellRet'])
rdf['F300s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet1'], rdf['sellRet1'])
rdf['mtm60s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet2'], rdf['sellRet2'])
rdf['mtm180s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet3'], rdf['sellRet3'])
rdf['mtm300s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet4'], rdf['sellRet4'])

rdf[['F90s_return', 'F300s_return', 'mtm60s_return', 'mtm180s_return', 'mtm300s_return']].describe()


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,F90s_return,F300s_return,mtm60s_return,mtm180s_return,mtm300s_return
count,6648.000000,6648.000000,6648.000000,6648.000000,6648.000000
mean,0.001133,0.000909,0.001215,0.001027,0.000911
std,0.003549,0.005769,0.002812,0.004627,0.005719
min,-0.021372,-0.027587,-0.012607,-0.035136,-0.027802
25%,-0.000726,-0.002109,-0.000270,-0.001352,-0.002034
50%,0.001013,0.000837,0.001096,0.001057,0.000844
75%,0.002901,0.003802,0.002636,0.003245,0.003793
max,0.027512,0.035992,0.025803,0.030075,0.034264


In [41]:
for i in ['fill_rate', 'cancel_rate']:
    f[i] = f[i].apply(lambda x: '%.1f%%'%(x*100))
for i in ['orderNtl', 'tradeNtl']:
    f[i] = (f[i]/1000).astype(int)
for i in ['cancel_duration_med', 'cancel_duration_90p']:
    f[i] = (f[i]/1000000).apply(lambda x: '%.0f'%(x))
for i in ['mrm_return', 'F90s_return', 'F300s_return', 'mrm_return_60s', 'mrm_return_180s', 'mrm_return_300s']:
     f[i] = f[i].apply(lambda x: '%.2f'%(x*10000))
f['stockGroup'] = np.where(f['order_type'].isin(['improved-NBBO make', 'on-NBBO make']), 'groupA', 'groupB')
f = f.rename(columns={'order_type':'orderType', 'fill_rate':'fillRate', 'cancel_rate':'cancelRate', 'mrm_return':'Mta', 'mrm_return_60s':'Mta60s', 'mrm_return_180s':'Mta180s', 'mrm_return_300s':'Mta300s'})
HTML(f.groupby(['duration', 'index_name', 'accCode', 'Side',  'stockGroup', 'orderType'])['orderNtl', 'tradeNtl', 'cancelRate', 'cancel_duration_med', 'cancel_duration_90p', 'fillRate', 'Mta', 'Mta60s', 'Mta180s', 'Mta300s', 'F90s_return', 'F300s_return'].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # This is added back by InteractiveShellApp.init_path()


In [34]:
HTML(f[f['duration'] == '20201221-20201224'].groupby(['duration', 'index_name', 'accCode', 'Side',  'stockGroup', 'orderType'])['orderNtl', 'tradeNtl', 'cancelRate', 'cancel_duration_med', 'cancel_duration_90p', 'fillRate', 'Mta', 'Mta60s', 'Mta180s', 'Mta300s', 'F90s_return', 'F300s_return'].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [35]:
f.groupby('orderType')['tradeNtl'].sum()

orderType
improved-NBBO make    52949
on-NBBO make          62374
take                  85829
Name: tradeNtl, dtype: int32